In [9]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import operator
import pickle

import src.fusiontools as fusiontools
import src.scoring as scoring
import src.postprocess as postprocess

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
path_to_results = 'results/'

In [3]:
# Load best recency model
recency_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
with open(recency_rank_dic_path, 'rb') as infile:
    recency_ranks = pickle.load(infile)

    
# Load best knn model
knn_rank_dic_path = 'ranks_test/knn-k-50-rec-all'
with open(knn_rank_dic_path, 'rb') as infile:
    knn_ranks = pickle.load(infile)


In [5]:
for values in knn_ranks.values():
    print(len(values))

151
67
140
1
242
3
8
272
33
56
211
57
294
15
141
220
34
217
33
77
82
47
54
62
146
47
92
91
726
110
112
157
272
51
302
63
183
183
33
68
254
24
24
368
14
60
97
103
340
8
47
80
143
1
52
43
47
78
233
33
68
183
360
42
52
86
79
77
6
85
214
174
93
47
72
100
47
3
537
53
75
150
14
58
79
86
80
126
156
49
131
73
170
47
69
81
272
52
175
98
90
81
138
70
136
73
107
116
75
48
107
130
326
214
271
82
66
70
65
109
96
153
199
63
109
94
71
73
132
106
61
40
99
309
80
81
238
42
73
41
551
135
250
32
69
112
52
54
62
54
242
29
104
46
42
44
276
181
24
52
120
129
60
66
62
94
7
166
40
147
48
72
85
154
147
150
72
325
356
89
48
129
30
45
54
154
64
111
13
93
101
18
107
106
61
53
135
65
89
106
209
139
6
145
35
81
203
82
75
86
88
39
93
39
256
46
97
48
17
8
67
210
70
181
57
17
38
43
24
129
46
162
123
92
104
46
111
181
52
66
88
62
47
98
53
92
55
87
39
47
170
103
73
69
99
57
272
135
271
69
150
225
60
49
65
85
26
787
76
69
80
48
87
53
70
62
40
26
269
134
57
263
26
32
55
73
67
88
102
74
357
209
16
74
70
73
75
74
69
72
75
2

In [16]:
models = [knn_ranks, recency_ranks]
weights = [0.20716, 0.39316]
reranking_coeff = 20
fusion_dic = fusiontools.reciprocal_rerank(models, reranking_coeff,  weights=weights)
print(fusion_dic)

            
                

{172034: ['james.d.steffes@enron.com', 'michael.tribolet@enron.com', 'elizabeth.sager@enron.com', 'rick.buy@enron.com', 'e..haedicke@enron.com', 'richard.b.sanders@enron.com', 's..bradford@enron.com', 'jeff.dasovich@enron.com', 'marcus.nettelton@enron.com', 'robert.eickenroht@enron.com'], 85335: ['pkuykendall@wm.com', 'jennifer_lamprecht@oxy.com', 'kuykendall@enron.com', 'andrewhescock@hotmail.com', 'susan.hadix@elpaso.com', 'cathy_morse@foleys.com', 'scottymorales@home.com', 'mike.grigsby@enron.com', 'drrichardson@cmsenergy.com', 'david.dronet@enron.com'], 40962: ['terry.kowalke@enron.com', 'john.buchanan@enron.com', 'lynn.blair@enron.com', 'ava.garcia@enron.com', 'steven.january@enron.com', 'raetta.zadow@enron.com', 'shelley.corman@enron.com', 'michael.bodnar@enron.com', 'larry.berger@enron.com', 'sheila.nacey@enron.com'], 118799: ['no.address@enron.com', 'steven.j.kean@enron.com', 'mark.palmer@enron.com', 'karen.denne@enron.com', 'vance.meyer@enron.com', 'meredith.philipp@enron.com'

In [17]:
postprocess.write_results_ranked(fusion_dic, path_to_results, 'fusion-weighted-reranking-20-tfdif-knn-recency.csv')